## LLM

In [1]:
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
from src.tokenizer import TokenizerBPE, word_split, normalize_to_ascii, pair_freq


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import numpy as np
import pickle as pkl
from tqdm.notebook import tqdm
from src.data_handling import read_first_n

# disable gpu for testing purposes
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [6]:
corpus = read_first_n('corpus/stories', 100000)

In [7]:
tokenizer = TokenizerBPE(corpus, 16000, lowercase=True)

Lowercasing corpus


  0%|          | 0/92579 [00:00<?, ?it/s]

Merging tokens


  0%|          | 0/16000 [00:00<?, ?it/s]

t h
i n
th e
a n
e r
o n
r e
a t
e n
o r
e d
s t
a l
t o
a r
in g
o u
i t
i s
e s
a s
o f
an d
i c
l e
i g
o m
i d
i on
i l
en t
s e
a c
h e
b e
ig h
a y
v e
a m
a d
f or
c h
r o
w h
' s
o l
l y
th at
i m
u n
o w
d e
s a
i r
v er
o t
u r
u t
t er
n e
igh t
w e
at ion
c on
p e
g e
a re
c e
w as
sa id
w i
il l
a b
o p
l d
h a
f r
s u
m o
t r
wi th
l i
er s
re s
c om
u s
k e
c t
t e
ou n
at e
l o
ou t
m ent
al l
h igh
l ight
s h
p ro
a g
i st
a p
ou r
u l
the r
a in
high light
f e
on e
b ut
p l
er e
s o
h is
ha ve
m e
g o
d ay
r i
m an
b y
l a
fr om
t s
p or
n ot
e x
it y
h as
p o
i f
a f
ar t
e ar
w or
ou ld
i es
th is
i al
a u
the y
ac k
0 0
en d
es s
om e
wh o
an t
g h
d o
b o
of f
g r
a k
q u
l l
i ve
ic e
por t
h er
w ill
i e
ne w
c o
s ay
c l
u st
y ear
u p
y ou
ac t
s he
i a
p r
ar d
c r
an s
t im
n n
p er
i v
p res
' t
c nn
n o
a st
in d
w ere
s p
the ir
n ow
b l
id e
f o
p ol
oun d
b r
h ad
c an
u d
e m
mo re
on g
ab out
b er
d er
re d
be en
e l
o ther
id ent
o k
s ome
ou gh
e v


In [ ]:
corpus = read_first_n('corpus/stories', 100)

In [8]:
tokenizer.destroy_hash()
pkl.dump(tokenizer, open("tokenizer_CNN16000_lowercase.pkl", "wb")) 

In [9]:
def tokenize(indices, merge_list):
    indices = np.array(indices)
    for pair, new_idx in merge_list:
        slice = np.where(np.logical_and(indices[:-1] == pair[0],  indices[1:] == pair[1]))
        if len(slice[0]) > 0:
            indices[:-1][slice] = new_idx
            indices = np.delete(indices, (slice[0]+1))

    return tf.expand_dims(tf.convert_to_tensor(indices, dtype=tf.int32), axis=0)

In [10]:
corpus = [line.lower() for line in tqdm(corpus)]

  0%|          | 0/92579 [00:00<?, ?it/s]

In [11]:
tokenizer = pkl.load(open("tokenizer_CNN16000_lowercase.pkl", "rb"))
tokenizer.create_hash()
indicies_list = [np.array(tokenizer.tokenizer.tokenize(line)) for line in tqdm(corpus)]

  0%|          | 0/92579 [00:00<?, ?it/s]

In [12]:
indicies_merged_list = [tokenize(indicies, tokenizer.merge_list) for indicies in tqdm(indicies_list)]

  0%|          | 0/92579 [00:00<?, ?it/s]

In [13]:
pkl.dump(indicies_merged_list, open('corpus/CNN_tokenized16000_lowercase.pkl', 'wb'))

## 